# AlexNet (Transfer Learning)

In [26]:
import torch
import torchvision.models as models
from torch import nn, optim
from torchvision import transforms
from torch.utils.data import DataLoader, random_split
from torchsummary import summary
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from PIL import Image
import matplotlib.pyplot as plt
from ptflops import get_model_complexity_info
import time

# Import custom utility module
from utils import get_transform, prepare_train_test_data, calculate_evaluation_metrics

In [27]:
# Device configuration (Select CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\n Device : {device}")


 Device : cuda


### Load and prepare the data

In [28]:
# Transform data
normalize_mean = [0.485, 0.456, 0.406]
normalize_std = [0.229, 0.224, 0.225]

image_input_size = (3, 227, 227) # Image input size for the model
transform = get_transform(resize_image_size = (227, 227), normalize_mean= normalize_mean, normalize_std = normalize_std)

# Prepare Train/Val/Test Data
train_loader, val_loader, test_loader = prepare_train_test_data(transform)

# Model keyword
model_keyword = "alexnet"

### Load pre-trained AlexNet model

In [37]:
# Load pre-trained AlexNet
model = models.alexnet(pretrained=True)

# Modify the final fully connected layer for binary classification
num_features = model.classifier[6].in_features  # Get the input features of the last layer
model.classifier[6] = nn.Linear(num_features, 2)  # Replaced the final layer for binary classification: Internal waves (1) or No waves (0). (This is actually treating binary classification as a 2-class problem when compared to a single output neuron)
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()

# optimizer = optim.Adam(model.parameters(), lr=1e-3)
optimizer = optim.SGD(model.parameters(), lr=1e-3, momentum=0.9) # AlexNet is sensitive to SGD compared to Adam + it doesn't use BatchNormalization as other newer models. Therefor, using the SGD optimizer.

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.1) # Implement a learning rate scheduler to adjust the learning rate during training
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)

# Print summary
summary(model, input_size=image_input_size)
print(f"Total number of layers: {sum(1 for _ in model.modules())}")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 56, 56]          23,296
              ReLU-2           [-1, 64, 56, 56]               0
         MaxPool2d-3           [-1, 64, 27, 27]               0
            Conv2d-4          [-1, 192, 27, 27]         307,392
              ReLU-5          [-1, 192, 27, 27]               0
         MaxPool2d-6          [-1, 192, 13, 13]               0
            Conv2d-7          [-1, 384, 13, 13]         663,936
              ReLU-8          [-1, 384, 13, 13]               0
            Conv2d-9          [-1, 256, 13, 13]         884,992
             ReLU-10          [-1, 256, 13, 13]               0
           Conv2d-11          [-1, 256, 13, 13]         590,080
             ReLU-12          [-1, 256, 13, 13]               0
        MaxPool2d-13            [-1, 256, 6, 6]               0
AdaptiveAvgPool2d-14            [-1, 25

### Train the model

In [38]:
# Initialize a DataFrame to log training/validation metrics
log_df = []

# Record the start time
start_time = time.time()

# Training loop
num_epochs = 50
best_val_loss = float('inf')  # Initialize best validation loss for saving model

for epoch in range(num_epochs):
    # Training phase
    model.train()
    train_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # Validation phase
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
    
    val_accuracy = val_correct / val_total * 100
    
    # Adjust learning rate
    scheduler.step(val_loss)
    
    # Calculate FLOPs after each epoch
    with torch.no_grad():
        flops, _ = get_model_complexity_info(model, (image_input_size), as_strings=False, print_per_layer_stat=False)

    # Log metrics into the DataFrame
    new_row = {
        "Epoch": epoch + 1,
        "Train_Loss": train_loss / len(train_loader),
        "Validation_Loss": val_loss / len(val_loader),
        "Validation_Accuracy": val_accuracy,
        "FLOPs": flops
    }
    log_df.append(new_row)

    # Print metrics for this epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss / len(train_loader):.4f}, "
          f"Val Loss: {val_loss / len(val_loader):.4f}, Val Accuracy: {val_accuracy:.2f}%, "
          f"GFLOPs: {(flops / 1e9 ):.2f}")

    # Save the best model based on validation loss
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), f"model_outputs_data/best_{model_keyword}_model.pth")


# Calculate the total training time in seconds
print(f"\nTotal Training Time: {(time.time() - start_time):.2f} seconds")

# Save the DataFrame to a CSV file for later use
log_df = pd.DataFrame(log_df)
log_df.to_csv(f"model_outputs_data/model_evaluation_logs/training_logs_{model_keyword}.csv", index=False)

Epoch [1/50], Train Loss: 0.4274, Val Loss: 0.3704, Val Accuracy: 82.82%, GFLOPs: 0.71
Epoch [2/50], Train Loss: 0.2876, Val Loss: 0.2743, Val Accuracy: 86.20%, GFLOPs: 0.71
Epoch [3/50], Train Loss: 0.2393, Val Loss: 0.2963, Val Accuracy: 86.93%, GFLOPs: 0.71
Epoch [4/50], Train Loss: 0.2581, Val Loss: 0.2290, Val Accuracy: 90.46%, GFLOPs: 0.71
Epoch [5/50], Train Loss: 0.1936, Val Loss: 0.2652, Val Accuracy: 88.69%, GFLOPs: 0.71
Epoch [6/50], Train Loss: 0.1630, Val Loss: 0.2590, Val Accuracy: 87.96%, GFLOPs: 0.71
Epoch [7/50], Train Loss: 0.1307, Val Loss: 0.2020, Val Accuracy: 91.19%, GFLOPs: 0.71
Epoch [8/50], Train Loss: 0.1190, Val Loss: 0.2237, Val Accuracy: 91.92%, GFLOPs: 0.71
Epoch [9/50], Train Loss: 0.0978, Val Loss: 0.2223, Val Accuracy: 92.07%, GFLOPs: 0.71
Epoch [10/50], Train Loss: 0.0994, Val Loss: 0.2557, Val Accuracy: 90.01%, GFLOPs: 0.71
Epoch [11/50], Train Loss: 0.0796, Val Loss: 0.2371, Val Accuracy: 91.63%, GFLOPs: 0.71
Epoch [12/50], Train Loss: 0.0750, Val Lo

### Prediction and Model evaluation

In [39]:
model.eval()
# Load the best model for testing
# model.load_state_dict(torch.load(f"model_outputs_data/best_{model_keyword}_model.pth"))

# Test the model
all_labels = []
all_preds = []
all_probs = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        probabilities = torch.softmax(outputs, dim=1)
        _, predicted = torch.max(outputs, 1)

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())
        all_probs.extend(probabilities[:, 1].cpu().numpy())

# Print evaluation metrics
calculate_evaluation_metrics(all_labels, all_preds, all_probs, model_keyword)


Test Accuracy: 93.63%
Precision: 0.94
Recall: 0.94
F1-Score: 0.94

Classification Report:
                   precision    recall  f1-score   support

No Internal Waves       0.93      0.94      0.94       719
   Internal Waves       0.94      0.94      0.94       740

         accuracy                           0.94      1459
        macro avg       0.94      0.94      0.94      1459
     weighted avg       0.94      0.94      0.94      1459


Confusion Matrix:
[[674  45]
 [ 48 692]]

 Predictions saved to model_outputs_data/model_prediction_logs/alexnet_labels_predictions.csv
